In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
import glob
import json
import os
from pprint import pprint

In [41]:
import multiprocessing as mp

In [8]:
import pandas as pd
import numpy as np
#import pyarrow
from scipy.special import exp10
from tslearn.clustering import TimeSeriesKMeans, KShape
from tslearn.metrics import dtw
from tslearn.clustering import silhouette_score
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [9]:
sys.path.append("./functions")

In [28]:
from helper_functions import percentile, sel_cloud_center
from clustering_functions import save_model, load_model, run_cluster
from preproc_functions import prep_cluster_data, prep_preprocessed_cluster_data, get_pred_dfs

## Load Data

In [11]:
# specify directory of data
data_dir = "/net/n2o/wolke_scratch2/kjeggle/CIRRUS_PIPELINE/merra_extended/DATA_CUBE/dataframes"

In [12]:
df = pd.read_feather(data_dir+"/cirrus_cloud_trajectories.ftr")

In [13]:
# create dataframe with only trajectory startpoints
start_points_df = df.query("timestep==0")

In [ ]:
# round temperature and omega → used for analysis and plotting
start_points_df["T_rounded"] = start_points_df["T"].round()
start_points_df["OMEGA_rounded"] = start_points_df["OMEGA"].round(2)

In [33]:
# add region variable
start_points_df.insert(len(start_points_df.columns), "region", "mid_latitudes")# traj_df.lat_top
start_points_df.loc[start_points_df.lat<30, "region"] = "tropics"
start_points_df.loc[start_points_df.lat>60, "region"] = "polar"

In [17]:
# use all cloud ids instead of only pretrained
cloud_ids = start_points_df.cloud_id.unique()[:

### Option I: create clustering data from scratch

In [20]:
### running this cell will take some time ###
# create dataframe and arrays used as clustering input
traj_df, traj_array, X = prep_cluster_data(df[df.cloud_id.isin(cloud_ids)],
                                        ["T_top","T_base"], 
                                        cloud_ids) 



### Option 2 (recommended): load preprocessed clustering data directly 

In [30]:
traj_df = pd.read_feather(data_dir+"/cluster_input_data.ftr")

In [31]:
traj_array, X = prep_preprocessed_cluster_data(traj_df, clustering_features=["T_top","T_base"])

## Fit Clustering Model

In [22]:
clustering_kws = {"metric":"dtw","max_iter":10,"max_iter_barycenter":10,"random_state":3,"tol":0.1}

### Run single clustering model for specified k 

In [23]:
n_cloud_ids = "all" # alternatively specify how many clouds should be used for clustering as int
k = 4 # number of clusters

In [24]:
if n_cloud_ids == "all":
    cluster_cloud_ids = cloud_ids
else:
    cluster_cloud_ids = np.random.choice(cloud_ids,size=n_cloud_ids,replace=False)

In [25]:
model =    run_cluster(k, 
                       X, 
                       cluster_cloud_ids,
                       clustering_kws,
                       model_id=f"cluster_model_k{k}",
                       model_path="cluster_models")


4 (10000, 24, 2)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    1.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 30000 out of 30000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 30000 out of 30000 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 30000 out of 30000 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 40000 out of 40000 | elapsed:    3.6s finished


4157.845 --> 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 40000 out of 40000 | elapsed:    3.6s finished


2292.730 --> 

...

2081.986 --> 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 40000 out of 40000 | elapsed:    3.6s finished


2080.822 --> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 40000 out of 40000 | elapsed:    3.6s finished


created: cluster_models/cluster_model_k4
saved model to file
saved cloud_ids to disk
saved model


In [26]:
model.labels_

array([0, 0, 0, ..., 2, 2, 1])

In [34]:
cluster_pred_df, traj_df = get_pred_dfs(start_points_df, 
                                        traj_df, 
                                        traj_array, 
                                        model.labels_,
                                        var="T")

/net/n2o/wolke/kjeggle/Repos/cloud_clustering/src/./functions/preproc_functions.py:40: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  traj_df = pd.merge(traj_df_, cluster_pred_df.drop(["trajectory_id_top","trajectory_id_base","T_rounded_base","T_rounded_top","OMEGA_rounded_base","OMEGA_rounded_top","DU_sup_log_base","DU_sup_log_top","nightday_flag_top","nightday_flag_base","region_top"],1), on="cloud_id")


In [35]:
cluster_pred_df.head(5)

,cloud_top_traj_id,cloud_id,cloud_base_traj_id,cloud_center_traj_id,cluster_idx,trajectory_id_top,dz_top_v2_top,lev_top,iwc_top,icnc_5um_top,...,icnc_5um_log_base,icnc_100um_log_base,reffcli_base,season_base,region_base,nightday_flag_base,T_rounded_base,OMEGA_rounded_base,DU_sup_log_base,DU_sub_log_base
0,200701_0,2007-01-03 04:00:000.0-33.251.0,200701_21,200701_10,0,200701_0,90.0,17280.0,1.069843,0.098097,...,-1.203616,-4.853872,19.221583,DJF,tropics,1.0,192.0,0.01,-5.443732,-4.053901
1,200701_1,2007-01-03 04:00:000.25-33.251.0,200701_22,200701_11,0,200701_1,90.0,17280.0,1.102443,0.098185,...,-0.841881,-3.581036,21.440011,DJF,tropics,1.0,192.0,0.00,-5.364338,-4.043067
2,200701_2,2007-01-03 04:00:000.5-33.251.0,200701_23,200701_12,0,200701_2,82.5,17280.0,2.186585,0.160492,...,-1.034875,-4.358526,19.989646,DJF,tropics,1.0,192.0,-0.00,-5.297240,-4.032497
3,200701_3,2007-01-03 04:00:000.75-33.251.0,200701_24,200701_13,0,200701_3,75.0,17280.0,2.412285,0.174655,...,-1.059946,-4.194467,20.038454,DJF,tropics,1.0,193.0,-0.01,-5.101329,-3.989623
4,200701_5,2007-01-03 04:00:001.25-33.01.0,200701_26,200701_15,0,200701_5,96.0,17280.0,1.620471,0.130176,...,-0.902290,-3.474469,20.655014,DJF,tropics,1.0,192.0,-0.01,-4.893218,-3.927620


In [36]:
traj_df.head(5)

,T_top,OMEGA_top,IWC_top,LWC_top,RH_ice_top,DU_sup_top,DU_sub_top,DU_sup_log_top,lonr_top,latr_top,...,iwc_base_y,icnc_5um_base_y,icnc_100um_base_y,iwc_log_base_y,icnc_5um_log_base_y,icnc_100um_log_base_y,reffcli_base_y,season_base_y,region,DU_sub_log_base_y
0,191.451,0.014,0.0,0.0,1.034866,0.000004,0.000060,-5.390179,-33.25,0.00,...,0.507544,0.062573,0.000014,-0.294526,-1.203616,-4.853872,19.221583,DJF,tropics,-4.053901
1,190.806,0.016,0.0,0.0,1.153714,0.000004,0.000061,-5.405313,-33.50,0.25,...,0.507544,0.062573,0.000014,-0.294526,-1.203616,-4.853872,19.221583,DJF,tropics,-4.053901
2,190.909,0.002,0.0,0.0,1.120918,0.000004,0.000062,-5.407863,-34.00,0.25,...,0.507544,0.062573,0.000014,-0.294526,-1.203616,-4.853872,19.221583,DJF,tropics,-4.053901
3,190.930,-0.008,0.0,0.0,1.116948,0.000005,0.000068,-5.282858,-34.25,0.50,...,0.507544,0.062573,0.000014,-0.294526,-1.203616,-4.853872,19.221583,DJF,tropics,-4.053901
4,190.638,-0.005,0.0,0.0,1.186994,0.000005,0.000068,-5.282858,-34.50,0.50,...,0.507544,0.062573,0.000014,-0.294526,-1.203616,-4.853872,19.221583,DJF,tropics,-4.053901


## Run for different cluster K in parallel

In [ ]:
if __name__ == '__main__':
    pool = mp.Pool(8)
    sum_of_squared_distances = []
    km_dbas = []
    K = range(2,9)
    for k in K:
        print(f"k: {k}")
        pool.apply_async(run_cluster, args=(k, X, cluster_cloud_ids, clustering_kws,f"cluster_model_k{k}"))
    pool.close()

## Vertical velocity clustering

for each temperature cluster, do another clustering along the vertical velocity at cloud center

In [37]:
clustering_kws = {"metric":"dtw","max_iter":10,"max_iter_barycenter":10,"random_state":3,"tol":0.01}

In [38]:
parent_cluster_pred_df = cluster_pred_df 

In [39]:
n_parent_clusters = parent_cluster_pred_df.cluster_idx.unique().shape[0]

In [48]:
if __name__ == '__main__':
    pool = mp.Pool(4)
    for cluster_idx in parent_cluster_pred_df.cluster_idx.unique():
        print(cluster_idx)
        # get cloud_ids of cluster
        cluster_cloud_ids = parent_cluster_pred_df.query(f"cluster_idx=={cluster_idx}").cloud_id.unique()
        # create clustering input data
        traj_df, traj_array, X = prep_cluster_data(df, ["OMEGA_center"], cloud_ids=cluster_cloud_ids)
        print(cluster_idx, X.shape)
        model_id = f"temp_k{n_parent_clusters}_{cluster_idx}_omega"
        pool.apply_async(run_cluster, args=(3, X, cluster_cloud_ids, clustering_kws, model_id))
    pool.close()

0
3 (1035, 24, 1)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1035 out of 1035 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0 (1035, 24, 1)
2


[Parallel(n_jobs=1)]: Done 3105 out of 3105 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3105 out of 3105 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3105 out of 3105 | elapsed:    0.2s finished


0.017 --> 

...

1.057 --> 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10542 out of 10542 | elapsed:    0.6s finished


1.050 --> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10542 out of 10542 | elapsed:    0.6s finished


created: ./cluster_models/all_4_temp_3_omega_all_data
saved model to file
saved cloud_ids to disk
saved model
